In [0]:
import pandas as pd
import numpy as np
import csv as csv

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.utils import shuffle

import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

addploy = True
plot_lc = 0

##  Loading data

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving titanic.zip to titanic (1).zip
User uploaded file "titanic.zip" with length 34757 bytes


In [0]:
!ls

sample_data  titanic.zip


In [0]:
!mkdir titanic
!unzip titanic.zip titanic/

mkdir: cannot create directory ‘titanic’: File exists
Archive:  titanic.zip
caution: filename not matched:  titanic/


In [4]:
!ls

 gender_submission.csv	 test.csv  'titanic (1).zip'   train.csv
 sample_data		 titanic    titanic.zip


In [3]:
!cp titanic.zip titanic
!cd titanic/
!unzip titanic.zip

/bin/bash: line 0: cd: titanic/: Not a directory
Archive:  titanic.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace gender_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: gender_submission.csv   


In [0]:
!ls titanic

titanic.zip


In [5]:
train_dataset = pd.read_csv("train.csv")
test_dataset = pd.read_csv("test.csv")
print("train_dataset shape:", train_dataset.shape, "test_dataset shape:", test_dataset.shape)
train_dataset.head()

train_dataset shape: (891, 12) test_dataset shape: (418, 11)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
print("Id is unique:") if train_dataset.PassengerId.nunique() == train_dataset.shape[0] else print("Nope")
print("Train and test sets are distinct") if len(np.intersect1d(train_dataset.PassengerId.values, test_dataset.PassengerId.values)) == 0 else print("Nope")

data_has_nan = False
if train_dataset.count().min() == train_dataset.shape[0] and test_dataset.shape[0]:
    print("We do have any missing values")
else:
    data_has_nan = True
    print("we have nan")

Id is unique:
Train and test sets are distinct
we have nan


In [7]:
dtype_df = train_dataset.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df.groupby("Column Type").aggregate("count").reset_index()

,Column Type,Count
0,int64,5
1,float64,2
2,object,5


In [8]:
dtype_df

,Count,Column Type
0,PassengerId,int64
1,Survived,int64
2,Pclass,int64
3,Name,object
4,Sex,object
5,Age,float64
6,SibSp,int64
7,Parch,int64
8,Ticket,object
9,Fare,float64


## Analyzing data

In [9]:
if data_has_nan == True:
    nas = pd.concat([train_dataset.isnull().sum(), test_dataset.isnull().sum()], axis=1, keys=["Train Dataset", "Test Dataset"])
    print("Nan in datasets")
    print(nas[nas.sum(axis=1) > 0])

Nan in datasets
          Train Dataset  Test Dataset
Age                 177          86.0
Cabin               687         327.0
Embarked              2           0.0
Fare                  0           1.0


In [10]:
print(train_dataset[["Pclass", "Survived"]].groupby(["Pclass"], as_index=False).mean().sort_values(by="Survived", ascending=False))

   Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363


In [11]:
print(train_dataset[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by="Survived", ascending=False))


      Sex  Survived
0  female  0.742038
1    male  0.188908


In [12]:
print(train_dataset[["SibSp", "Survived"]].groupby(["SibSp"], as_index=False).mean().sort_values(by="Survived", ascending=False))

   SibSp  Survived
1      1  0.535885
2      2  0.464286
0      0  0.345395
3      3  0.250000
4      4  0.166667
5      5  0.000000
6      8  0.000000


In [13]:
print(train_dataset[['Parch', 'Survived']].groupby(["Parch"], as_index=False).mean().sort_values(by='Survived', ascending=False))

   Parch  Survived
3      3  0.600000
1      1  0.550847
2      2  0.500000
0      0  0.343658
5      5  0.200000
4      4  0.000000
6      6  0.000000


In [14]:
train_random_ages = np.random.randint(train_dataset["Age"].mean() - train_dataset["Age"].std(), train_dataset["Age"].mean() + train_dataset["Age"].std(), size=train_dataset["Age"].isnull().sum())
test_random_ages = np.random.randint(test_dataset["Age"].mean() - test_dataset["Age"].std(), test_dataset['Age'].mean() + test_dataset["Age"].std(), size=test_dataset["Age"].isnull().sum())

train_dataset["Age"][np.isnan(train_dataset["Age"])] = train_random_ages
test_dataset["Age"][np.isnan(test_dataset["Age"])] = test_random_ages
train_dataset["Age"] = train_dataset["Age"].astype(int)
test_dataset["Age"] = test_dataset["Age"].astype(int)

train_dataset["Embarked"].fillna("S", inplace=True)
test_dataset["Embarked"].fillna("S", inplace=True)
train_dataset["Port"] = train_dataset["Embarked"].map({"S": 0, "C": 1, "Q": 2}).astype(int)
test_dataset["Port"] = test_dataset["Embarked"].map({"S": 0, "C": 1, "Q": 2}).astype(int)

test_dataset["Fare"].fillna(test_dataset["Fare"].median(), inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


### Engineer New Features

In [0]:
train_dataset["Has_Cabin"] = train_dataset["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test_dataset["Has_cabin"] = test_dataset["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

full_dataset = [train_dataset, test_dataset]

for dataset in full_dataset:
    dataset["FamilySize"] = dataset["SibSp"] + dataset["Parch"] + 1
    
for dataset in full_dataset:
    dataset["IsAlone"] = 0
    dataset.loc[dataset["FamilySize"] == 1, "IsAlone"] = 1
    
# getting titles from names
train_dataset["Title"] = train_dataset.Name.str.extract('([A-Za-z]+)\.', expand=False)
test_dataset["Title"] = test_dataset.Name.str.extract('([A-Za-z]+)\.', expand=False)

for dataset in full_dataset:
    dataset["Title"] = dataset["Title"].replace(["Lady", "Countess", "Capt", "Col", "Don", "Dr", "Major", "Rev", "Sir", "Jonkheer", "Dona"], "Rare")
    dataset["Title"] = dataset["Title"].replace("Mlle", "Miss")
    dataset["Title"] = dataset["Title"].replace("Ms", "Miss")
    dataset["Title"] = dataset["Title"].replace("Mme", "Mrs")
    
## create new column "FamilySizeGroup" and assign "Alone", "Small", "big"
for dataset in full_dataset:
    dataset["FamilySizeGroup"] = "Small"
    dataset.loc[dataset["FamilySize"] == 1, 'FamilySizeGroup'] = "Alone"
    dataset.loc[dataset["FamilySize"] >= 5, 'FamilySizeGroup'] = "Big"
    
train_dataset[["FamilySize", "Survived"]].groupby(["FamilySize"], as_index=False).mean()

for dataset in full_dataset:
    dataset["Sex"] = dataset["Sex"].map({"female": 1, 'male': 0}).astype(int)
    
for dataset in full_dataset:
    dataset.loc[dataset["Age"] <= 14, "Age"] = 0
    dataset.loc[(dataset["Age"] > 32) & (dataset["Age"] <= 48), "Age"] = 2
    dataset.loc[(dataset["Age"] > 48) & (dataset["Age"] <= 64), "Age"] = 3
    dataset.loc[dataset["Age"] > 64, "Age"] = 4
    
for dataset in full_dataset:
    dataset.loc[dataset["Fare"] <= 7.91, "Fare"] = 0
    dataset.loc[(dataset["Fare"] > 7.91) & (dataset["Fare"] <= 31), "Fare"] = 2
    dataset.loc[dataset['Fare'] > 31, "Fare"] = 3
    dataset["fare"] = dataset["Fare"].astype(int)

In [17]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
family_mapping= {"Small": 0, "Alone": 1, "Big": 2}
for dataset in full_dataset:
    dataset["Title"] = dataset["Title"].map(title_mapping)
    dataset["FamilySizeGroup"] = dataset["FamilySizeGroup"].map(family_mapping)
    
for dataset in full_dataset:
    dataset["IsChildandRich"] = 0
    dataset.loc[(dataset["Age"] <= 0) & (dataset["Pclass"] == 1), 'IsChildandRich'] = 1
    dataset.loc[(dataset["Age"] <= 0) & (dataset['Pclass'] == 2), 'IsChildandRich'] = 1
    
for data in full_dataset:
    data["Cabin"] = data["Cabin"].fillna("X")
    data["Cabin"] = data["Cabin"].apply(lambda x: str(x)[0])
    data["Cabin"] = data["Cabin"].replace(["A", "D", "E", "T"], "M")
    data["Cabin"] = data["Cabin"].replace(["B", "C"], "H")
    data["Cabin"] = data["Cabin"].replace(["F", "G"], "L")
    data["Cabin"] = data["Cabin"].map({"X": 0, "L": 1, "M": 2, "H": 3}).astype(int)
    #data['Cabin'].loc[~data['Cabin'].isnull()] = 1
    #data['Cabin'].loc[data['Cabin'].isnull()] = 0

    
print("Finished data cleaning")

Finished data cleaning


In [18]:
print("train dataset: %s, test dataset: %s" %(str(train_dataset.shape), str(test_dataset.shape)))
train_dataset.head()

train dataset: (891, 20), test dataset: (418, 19)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Port,Has_Cabin,FamilySize,IsAlone,Title,FamilySizeGroup,fare,IsChildandRich
0,1,0,3,"Braund, Mr. Owen Harris",0,22,1,0,A/5 21171,0.0,0,S,0,0,2,0,NaN,NaN,0,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,2,1,0,PC 17599,3.0,3,C,1,1,2,0,NaN,NaN,3,0
2,3,1,3,"Heikkinen, Miss. Laina",1,26,0,0,STON/O2. 3101282,2.0,0,S,0,0,1,1,NaN,NaN,2,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,2,1,0,113803,3.0,3,S,0,1,2,0,NaN,NaN,3,0
4,5,0,3,"Allen, Mr. William Henry",0,2,0,0,373450,2.0,0,S,0,0,1,1,NaN,NaN,2,0


In [0]:
del train_dataset["PassengerId"]
del train_dataset["Name"]
del test_dataset["Name"]
del train_dataset["SibSp"]
del test_dataset["SibSp"]
del train_dataset["Parch"]
del test_dataset["Parch"]
del train_dataset['FamilySize']
del test_dataset['FamilySize']
del train_dataset["Cabin"]
del test_dataset["Cabin"]
del train_dataset["Ticket"]
del test_dataset["Ticket"]
del train_dataset["Port"]
del test_dataset["Port"]
del train_dataset["Embarked"]
del test_dataset["Embarked"]

In [20]:
print("train dataset: %s, test dataset: %s" %(str(train_dataset.shape), str(test_dataset.shape)))
train_dataset.head()

train dataset: (891, 11), test dataset: (418, 11)


,Survived,Pclass,Sex,Age,Fare,Has_Cabin,IsAlone,Title,FamilySizeGroup,fare,IsChildandRich
0,0,3,0,22,0.0,0,0,NaN,NaN,0,0
1,1,1,1,2,3.0,1,0,NaN,NaN,3,0
2,1,3,1,26,2.0,0,1,NaN,NaN,2,0
3,1,1,1,2,3.0,1,0,NaN,NaN,3,0
4,0,3,0,2,2.0,0,1,NaN,NaN,2,0


In [21]:
X_train = train_dataset.drop("Survived", axis=1)
Y_train = train_dataset["Survived"]
X_test = test_dataset.drop("PassengerId", axis=1).copy()

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)

(891, 10)
(891,)
(418, 10)


In [0]:
for dataset in full_dataset:
    dataset["FamilySizeGroup"] = "Small"
    dataset.loc[dataset["FamilySize"] == 1, 'FamilySizeGroup'] = "Alone"
    dataset.loc[dataset["FamilySize"] >= 5, 'FamilySizeGroup'] = "Big"

In [0]:
del X_test['Title']
del X_test['FamilySizeGroup']

In [62]:
X_test.dtypes

Pclass            int64
Sex               int64
Age               int64
Fare              int64
Has_cabin         int64
IsAlone           int64
fare              int64
IsChildandRich    int64
Has_Cabin         int64
dtype: object

In [47]:
print(X_train.shape, X_test.shape)

(891, 8) (418, 8)


In [50]:
Y_train.dtypes

dtype('int64')

In [64]:
all_data = pd.concat((X_train, X_test), ignore_index=True)
all_data.dtypes

Age               int64
Fare              int64
Has_Cabin         int64
IsAlone           int64
IsChildandRich    int64
Pclass            int64
Sex               int64
fare              int64
dtype: object

In [0]:
del X_test['Has_cabin']

In [0]:
X_test['Has_Cabin'] = X_test['Has_cabin']

In [65]:
## try polynomial

from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures

addpoly = True
if addpoly:
    all_data = pd.concat((X_train, X_test), ignore_index=True)
    all_data["Has_Cabin"] = all_data["Has_Cabin"].astype(int)
    scaler = MinMaxScaler()
    scaler.fit(all_data)
    all_data = scaler.transform(all_data)
    poly = PolynomialFeatures(2)
    all_data = poly.fit_transform(all_data)
    
    X_train = all_data[:train_dataset.shape[0]]
    X_test = all_data[train_dataset.shape[0]:]
    
    print(X_train.shape)
    print(Y_train.shape)
    print(X_test.shape)

(891, 45)
(891,)
(418, 45)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [67]:
## Random Forests

random_forest = RandomForestClassifier(criterion='gini', n_estimators=1000, min_samples_split=10,
                                        min_samples_leaf=1, max_features="auto", oob_score=True, 
                                         random_state=1, n_jobs=-1)

seed = 43
random_forest = RandomForestClassifier(n_estimators=1000, criterion='entropy', max_depth=5, 
                                        min_samples_split=2, n_jobs=-1, random_state=seed, verbose=0)

random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)

result_train = random_forest.score(X_train, Y_train)
result_val = cross_val_score(random_forest, X_train, Y_train, cv=5).mean()

print("Taring score = %s, while validation score = %s" %(result_train, result_val))

Taring score = 0.8338945005611672, while validation score = 0.8047558202091152


In [68]:
submission = pd.DataFrame({
        "PassengerId": test_dataset["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('titanic.csv', index=False)
print('Exported')

Exported
